In [2]:
def dm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MSE", power=2):
    # Routine for checking errors
    def error_check():
        rt = 0
        msg = ""
        # Check if h is an integer
        if (not isinstance(h, int)):
            rt = -1
            msg = "The type of the number of steps ahead (h) is not an integer."
            return (rt, msg)
        # Check the range of h
        if (h < 1):
            rt = -1
            msg = "The number of steps ahead (h) is not large enough."
            return (rt, msg)
        len_act = len(actual_lst)
        len_p1 = len(pred1_lst)
        len_p2 = len(pred2_lst)
        # Check if lengths of actual values and predicted values are equal
        if (len_act != len_p1 or len_p1 != len_p2 or len_act != len_p2):
            rt = -1
            msg = "Lengths of actual_lst, pred1_lst and pred2_lst do not match."
            return (rt, msg)
        # Check range of h
        if (h >= len_act):
            rt = -1
            msg = "The number of steps ahead is too large."
            return (rt, msg)
        # Check if criterion supported
        if (crit != "MSE" and crit != "MAPE" and crit != "MAD" and crit != "poly"):
            rt = -1
            msg = "The criterion is not supported."
            return (rt, msg)
            # Check if every value of the input lists are numerical values
        from re import compile as re_compile
        comp = re_compile("^\d+?\.\d+?$")

        def compiled_regex(s):
            """ Returns True is string is a number. """
            if comp.match(s) is None:
                return s.isdigit()
            return True

        for actual, pred1, pred2 in zip(actual_lst, pred1_lst, pred2_lst):
            is_actual_ok = compiled_regex(str(abs(actual)))
            is_pred1_ok = compiled_regex(str(abs(pred1)))
            is_pred2_ok = compiled_regex(str(abs(pred2)))
            if (not (is_actual_ok and is_pred1_ok and is_pred2_ok)):
                msg = "An element in the actual_lst, pred1_lst or pred2_lst is not numeric."
                rt = -1
                return (rt, msg)
        return (rt, msg)

    # Error check
    error_code = error_check()
    # Raise error if cannot pass error check
    if (error_code[0] == -1):
        raise SyntaxError(error_code[1])
        return
    # Import libraries
    from scipy.stats import t
    import collections
    import pandas as pd
    import numpy as np

    # Initialise lists
    e1_lst = []
    e2_lst = []
    d_lst = []

    # convert every value of the lists into real values
    actual_lst = pd.Series(actual_lst).apply(lambda x: float(x)).tolist()
    pred1_lst = pd.Series(pred1_lst).apply(lambda x: float(x)).tolist()
    pred2_lst = pd.Series(pred2_lst).apply(lambda x: float(x)).tolist()

    # Length of lists (as real numbers)
    T = float(len(actual_lst))

    # construct d according to crit
    if (crit == "MSE"):
        for actual, p1, p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append((actual - p1) ** 2)
            e2_lst.append((actual - p2) ** 2)
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAD"):
        for actual, p1, p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append(abs(actual - p1))
            e2_lst.append(abs(actual - p2))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "MAPE"):
        for actual, p1, p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append(abs((actual - p1) / actual))
            e2_lst.append(abs((actual - p2) / actual))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)
    elif (crit == "poly"):
        for actual, p1, p2 in zip(actual_lst, pred1_lst, pred2_lst):
            e1_lst.append(((actual - p1)) ** (power))
            e2_lst.append(((actual - p2)) ** (power))
        for e1, e2 in zip(e1_lst, e2_lst):
            d_lst.append(e1 - e2)

            # Mean of d
    mean_d = pd.Series(d_lst).mean()

    # Find autocovariance and construct DM test statistics
    def autocovariance(Xi, N, k, Xs):
        autoCov = 0
        T = float(N)
        for i in np.arange(0, N - k):
            autoCov += ((Xi[i + k]) - Xs) * (Xi[i] - Xs)
        return (1 / (T)) * autoCov

    gamma = []
    for lag in range(0, h):
        gamma.append(autocovariance(d_lst, len(d_lst), lag, mean_d))  # 0, 1, 2
    V_d = (gamma[0] + 2 * sum(gamma[1:])) / T
    DM_stat = V_d ** (-0.5) * mean_d
    harvey_adj = ((T + 1 - 2 * h + h * (h - 1) / T) / T) ** (0.5)
    DM_stat = harvey_adj * DM_stat
    # Find p-value
    p_value = 2 * t.cdf(-abs(DM_stat), df=T - 1)

    # Construct named tuple for return
    dm_return = collections.namedtuple('dm_return', 'DM p_value')

    rt = dm_return(DM=DM_stat, p_value=p_value)

    return rt


if __name__ == '__main__':
    # 使用
    import random
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    from pandas import read_csv
    import math

    # load the dataset
    dataframe = read_csv('C:/Users/Administrator/Desktop/XI-2/data/try2ed/DM_data60.csv', engine='python')
    #print(dataframe)
    #print("数据集的长度：",len(dataframe))
    dataset1 = dataframe.values
    # 将整型变为float
    dataset1 = dataset1.astype('float32')
    dataset1=dataset1[:,0:9]
   
    actual30 = dataset1[:,0]
    actual30 = actual30.reshape(-1)
    pred30_1 = dataset1[:,1]
    pred30_1 = pred30_1.reshape(-1)
    pred30_2 = dataset1[:,2]
    pred30_2 = pred30_2.reshape(-1)
    pred30_3 = dataset1[:,3]
    pred30_3 = pred30_3.reshape(-1)
    pred30_4 = dataset1[:,4]
    pred30_4 = pred30_4.reshape(-1)
    pred30_5 = dataset1[:,5]
    pred30_5 = pred30_5.reshape(-1)
    pred30_6 = dataset1[:,6]
    pred30_6 = pred30_6.reshape(-1)
    pred30_7 = dataset1[:,7]
    pred30_7 = pred30_7.reshape(-1)
    pred30_8 = dataset1[:,8]
    pred30_8 = pred30_8.reshape(-1)


    rtc30_2 = dm_test(actual30, pred30_1, pred30_2, h=1, crit="MAD")
    print(rtc30_2)
    rtc30_3 = dm_test(actual30, pred30_1, pred30_3, h=1,crit="MAD")  
    print(rtc30_3)
    rtc30_4 = dm_test(actual30, pred30_1, pred30_4, h=1,crit="MAD") 
    print(rtc30_4)
    rtc30_5 = dm_test(actual30, pred30_1, pred30_5, h=1, crit="MAD")
    print(rtc30_5)
    rtc30_6 = dm_test(actual30, pred30_1, pred30_6, h=1, crit="MAD")
    print(rtc30_6)
    rtc30_7 = dm_test(actual30, pred30_1, pred30_7, h=1,crit="MAD")  
    print(rtc30_7)
    rtc30_8 = dm_test(actual30, pred30_1, pred30_8, h=1,crit="MAD") 
    print(rtc30_8)
    print("-------__________-------")

    #rt = dm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="MSE")
   # print(rt)
   # rt = dm_test(actual_lst, pred1_lst, pred2_lst, h=1, crit="poly", power=4)
   # print(rt)


dm_return(DM=-2.466222623829278, p_value=0.016301475358711348)
dm_return(DM=-4.62519811208411, p_value=1.8333568276912075e-05)
dm_return(DM=-5.223913858544937, p_value=1.9786466318527575e-06)
dm_return(DM=-5.007735198936931, p_value=4.47391710810947e-06)
dm_return(DM=-6.182159035310688, p_value=4.6948198420680536e-08)
dm_return(DM=-11.681998048351048, p_value=1.2831433561469756e-17)
dm_return(DM=-9.214989212044737, p_value=2.0750696872091704e-13)
-------__________-------


In [ ]:
random.seed(0)
actual_lst = actual30
pred1_lst = range(0, 100)
pred2_lst = range(0, 100)

actual_lst = random.sample(actual_lst, 100)
pred1_lst = random.sample(pred1_lst, 100)
pred2_lst = random.sample(pred2_lst, 100)
pred3_lst = actual_lst
print(actual30)

In [ ]:
print(actual30)